In [1]:
import numpy as np
obj_state = np.load("obj_keyframes.npy")
tip_poses = np.load("tip_poses.npy")
new_tip_poses = tip_poses.copy()

In [2]:
import pytorch_kinematics as pk
import torch

In [3]:
def getRelativePose(tip_pose, obj_pose, obj_orn):
    r = tip_pose - obj_pose
    orn = torch.tensor([obj_orn[3], obj_orn[0], obj_orn[1], obj_orn[2]])
    orn_inv = pk.quaternion_invert(orn)
    r = pk.quaternion_apply(orn_inv, torch.from_numpy(r)).numpy()
    return r

def getWorldPose(rel_pos, obj_pose, obj_orn):
    orn = torch.tensor([obj_orn[3], obj_orn[0], obj_orn[1], obj_orn[2]])
    r = pk.quaternion_apply(orn, torch.from_numpy(rel_pos)).numpy()
    return r + obj_pose

In [4]:
mid_0 = tip_poses[0,2]
mid_0_rel = getRelativePose(mid_0, obj_state[0,:3], obj_state[0,3:])
mid_1_world = getWorldPose(mid_0_rel, obj_state[1,:3], obj_state[1,3:])
new_tip_poses[1,2] = mid_1_world

thb_2 = tip_poses[2,0]
thb_2_rel = getRelativePose(thb_2, obj_state[2,:3], obj_state[2,3:])
thb_3_world = getWorldPose(thb_2_rel, obj_state[3,:3],obj_state[3,3:])
new_tip_poses[3,0] = thb_3_world

idx_3 = tip_poses[3,1]
idx_3_rel = getRelativePose(idx_3, obj_state[3,:3], obj_state[3,3:])
idx_4_world = getWorldPose(idx_3_rel, obj_state[4,:3],obj_state[4,3:])
new_tip_poses[4,1] = idx_4_world

In [5]:
np.save("new_tip_poses.npy", new_tip_poses)